# Creating housing archetypes using K-means method
### Q4 2020-21

## Introduction
This workbook will ...

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from scipy import stats
from kneed import KneeLocator 
from sklearn.datasets import make_blobs 
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import LocalOutlierFactor
from sklearn.model_selection import ParameterGrid
from itertools import permutations
from itertools import combinations

## Data Source
The data represents a subset of ERS records used for initial analysis.

In [ ]:
ers_sample_records = pd.read_csv(r"C:\Users\owner\Documents\NRCan\code\practice\InitialHousingData.csv",)
ers_sample_records 

#original amount of obsevations/rows 
og_obs = ers_sample_records.shape[0]

In [ ]:
ers_sample_records

In [ ]:
ers_sample_records.describe()

## Database Preperation and Varibable Selection 


### Select variables for clustering
Placeholder for this process. 

In [ ]:
# Air50P selected for 1D clustering test
cl_variables = ['MainWallIns']
test_data = ers_sample_records[cl_variables]


### Missing values

In [ ]:
#Remove rows with blank values
test_data_cleaned = test_data.dropna()
#display how many rows removed
test_data_cleaned.describe()

###  Inconsistent data removal 
Dependant on the variable. Example: floor area could not be negavtive.


### Ensure all values are numerical
K-means can only handle numerical values. 

In [ ]:
# encode catagorical variables 


### Remove outliers

K-means is based off finding the mean of clusters and since means are senstive to outliers so is k-means. If outliers are not delt with the they can have a large infulence in the clustering porcess that could result in poor partiotns that are not representaive of the data. Because of this outliers need to be removed properly. 

Three different outlier removal techniques are tested: local outlier factor (LOF), z-score, and inter quartile range (IQR). 

The data will be interperated before and after outlier removal though box plots( and scatter?) and statisical measures.

In [ ]:
#box plot before outlier removal
test_data_cleaned.boxplot() 

#### Local outlier factor
Local outlier factor (LOF) values identify an outlier based on the local neighborhood. It gives better results than the global approach to find outliers. A point will be considered as an outlier if it is at a small distance to the extremely dense cluster. 

In [ ]:
#define the model

lof = LocalOutlierFactor()
lof_pred = lof.fit_predict(test_data_cleaned) 

#extract the negative outputs as the outliers.
mask = lof_pred != -1

#remove rows with outliers 
lof_data = test_data_cleaned[mask] #i dont think this works for sets with more than 1 varibale
print([lof_data])

#print amount of points deleated

outliers_rem = og_obs - lof_data.shape[0]
print ('amount of outliers removed: %d' %outliers_rem )

#plot without after removal

lof_data.boxplot()

#### Z-score 

In [ ]:
#find absolute value of z-score for each observation
z = np.abs(stats.zscore(test_data_cleaned))

#only keep rows in dataframe with all z-scores less than absolute value of 3 
z_data = test_data_cleaned[(z<3).all(axis=1)]

#print amount of outliers removed
z_data

outliers_rem = og_obs - z_data.shape[0]
print ('amount of outliers removed: %d' %outliers_rem )

In [ ]:
#plotafter outlier removal
z_data.boxplot()

#### IQR
The IQR can be used to identify outliers by defining limits on the sample values that are a factor k of the IQR below the 25th percentile or above the 75th percentile. The common value for the factor k is the value 1.5 (used below). A factor k of 3 or more can be used to identify values that are extreme outliers or “far outs” when described in the context of box and whisker plots.

In [ ]:
#find Q1, Q3, and interquartile range for each column
Q1 =test_data_cleaned.quantile(q=.25)
Q3 = test_data_cleaned.quantile(q=.75)
IQR =test_data_cleaned.apply(stats.iqr)

#only keep rows in dataframe that have values within 1.5*IQR of Q1 and Q3
iqr_data = test_data_cleaned[~((test_data_cleaned < (Q1-1.5*IQR)) | (test_data_cleaned > (Q3+1.5*IQR))).any(axis=1)]

#print amount of outliers removed

outliers_rem = og_obs - iqr_data.shape[0]
print ('amount of outliers removed: %d' %outliers_rem )

In [ ]:
#plot of IQR outlier removal
iqr_data.boxplot()


 Mahalanobis distance - calc when multi variabale later
 https://www.statology.org/mahalanobis-distance-python/

### Clustering
Find the best mix of parameters for clustering. Parameters include oulier removal, scaling, initalization, and k.

In [ ]:
#import validation metrics 
#could also use package validclust
from sklearn.metrics import (silhouette_score, calinski_harabasz_score, davies_bouldin_score)

In [ ]:
# create sets of pre-processed data ready for clustering 
scaled_data_sets = [iqr_data,lof_data, z_data]


In [ ]:
#list of scalers
standard = StandardScaler()
minimax = MinMaxScaler()
scalers = [standard, minimax]
#list of initalizers 
r = 'random'
plus = 'k-means++' 
initalizer = [r, plus]



In [ ]:
# datasets being transfromed with standard scaler and random init
standard_rand_in=[]
standard_rand_cal=[]
standard_rand_dav=[]
standard_rand_sil=[]

for x in range(len(scaled_data_sets)):
    #first set of data to scale
    scaled_features = scalers[0].fit_transform(scaled_data_sets[x]) #not going to work for if sets change
   
    #determine amount of clusters elbow elblw method
    
    sse=[] #determine SSE for 1 to 11 clusters
    
    kmeans_kwargs = {
    "init":"random",  #for random 
    "n_init":10, 
    "max_iter":300,
    "random_state":42,}
    
    for k in range(1,11): #pre determined amount
        kmeans = KMeans(n_clusters=k, **kmeans_kwargs)
        kmeans.fit(scaled_features)
        sse.append(kmeans.inertia_)
    k1 = KneeLocator(range(1,11),sse,curve="convex", direction="decreasing")
    clamount=k1.elbow
    
    #cluster with amount of clusters 
    
    kmeans = KMeans(
    init="random",
    n_clusters= clamount,
    n_init=10,
    max_iter=300,
    random_state=42)
    
    kmeans.fit(scaled_features)
    
    standard_rand_in.append(kmeans.inertia_) # can use other validations too 
    
    pred_labels = kmeans.labels_
    
    sil = silhouette_score(scaled_data_sets[x], pred_labels)
    cal_score = calinski_harabasz_score(scaled_data_sets[x], pred_labels)
    dav_score = davies_bouldin_score(scaled_data_sets[x], pred_labels)
    
    standard_rand_cal.append(cal_score)
    standard_rand_dav.append(dav_score)
    standard_rand_sil.append(sil)

In [ ]:
#datasets being transfromed with mini scaler and random init
mini_rand_in=[]
mini_rand_cal=[]
mini_rand_sil=[]
mini_rand_dav=[]
for x in range(len(scaled_data_sets)):
    #first set of data to scale
    scaled_features = scalers[1].fit_transform(scaled_data_sets[x]) #not going to work for if sets change
   
    #determine amount of clusters elbow elblw method
    
    sse=[] #determine SSE for 1 to 11 clusters
    
    kmeans_kwargs = {
    "init":"random",  #for random 
    "n_init":10, 
    "max_iter":300,
    "random_state":42,}
    
    for k in range(1,11): #pre determined amount
        kmeans = KMeans(n_clusters=k, **kmeans_kwargs)
        kmeans.fit(scaled_features)
        sse.append(kmeans.inertia_)
    k1 = KneeLocator(range(1,11),sse,curve="convex", direction="decreasing")
    clamount=k1.elbow
    
    #cluster with amount of clusters 
    
    kmeans = KMeans(
    init="random",
    n_clusters= clamount,
    n_init=10,
    max_iter=300,
    random_state=42)
    
    kmeans.fit(scaled_features)
   
    #validation metrics 
    
    mini_rand_in.append(kmeans.inertia_) # can use other validations too 
   
    pred_labels = kmeans.labels_
    
    sil = silhouette_score(scaled_data_sets[x], pred_labels)
    cal_score = calinski_harabasz_score(scaled_data_sets[x], pred_labels)
    dav_score = davies_bouldin_score(scaled_data_sets[x], pred_labels)

    mini_rand_cal.append(cal_score)
    mini_rand_sil.append(sil)
    mini_rand_dav.append(dav_score)

In [ ]:
#datasets being transfromed with mini scaler and k++ init

mini_plus_in=[]
mini_plus_cal=[]
mini_plus_sil=[]
mini_plus_dav=[]

for x in range(len(scaled_data_sets)):
    #first set of data to scale
    scaled_features = scalers[1].fit_transform(scaled_data_sets[x]) #not going to work for if sets change
   
    #determine amount of clusters elbow elblw method
    
    sse=[] #determine SSE for 1 to 11 clusters
    
    kmeans_kwargs = {
    "init":"k-means++",  #for random 
    "n_init":10, 
    "max_iter":300,
    "random_state":42,}
    
    for k in range(1,11): #pre determined amount
        kmeans = KMeans(n_clusters=k, **kmeans_kwargs)
        kmeans.fit(scaled_features)
        sse.append(kmeans.inertia_)
    k1 = KneeLocator(range(1,11),sse,curve="convex", direction="decreasing")
    clamount=k1.elbow
    
    #cluster with amount of clusters 
    
    kmeans = KMeans(
    init="k-means++",
    n_clusters= clamount,
    n_init=10,
    max_iter=300,
    random_state=42)
    
    kmeans.fit(scaled_features)
    
    mini_plus_in.append(kmeans.inertia_) # can use other validations too 
    
    pred_labels = kmeans.labels_
    
    sil = silhouette_score(scaled_data_sets[x], pred_labels)
    cal_score = calinski_harabasz_score(scaled_data_sets[x], pred_labels)
    dav_score = davies_bouldin_score(scaled_data_sets[x], pred_labels)
  
    mini_plus_cal.append(cal_score)
    mini_plus_sil.append(sil)
    mini_plus_dav.append(dav_score)
    

  

In [ ]:
#datasets being transfromed with standard scaler and k++ init

standard_plus_in=[]
standard_plus_cal=[]
standard_plus_dav=[]
standard_plus_sil=[]


for x in range(len(scaled_data_sets)):
    #first set of data to scale
    scaled_features = scalers[0].fit_transform(scaled_data_sets[x]) #not going to work for if sets change
   
    #determine amount of clusters elbow elbow method
    
    sse=[] #determine SSE for 1 to 11 clusters
    
    kmeans_kwargs = {
    "init":"k-means++",  #for random 
    "n_init":10, 
    "max_iter":300,
    "random_state":42,}
    
    for k in range(1,11): #pre determined amount
        kmeans = KMeans(n_clusters=k, **kmeans_kwargs)
        kmeans.fit(scaled_features)
        sse.append(kmeans.inertia_)
    k1 = KneeLocator(range(1,11),sse,curve="convex", direction="decreasing")
    clamount=k1.elbow
    
    #cluster with amount of clusters 
    
    kmeans = KMeans(
    init="k-means++",
    n_clusters= clamount,
    n_init=10,
    max_iter=300,
    random_state=42)
    
    kmeans.fit(scaled_features)
    
    standard_plus_in.append(kmeans.inertia_) # can use other validations too 
     
    pred_labels = kmeans.labels_
    
    sil = silhouette_score(scaled_data_sets[x], pred_labels)
    cal_score = calinski_harabasz_score(scaled_data_sets[x], pred_labels)
    dav_score = davies_bouldin_score(scaled_data_sets[x], pred_labels)
    
    standard_plus_cal.append(cal_score)
    standard_plus_dav.append(dav_score)
    standard_plus_sil.append(sil)
  

In [ ]:
#intertia results
results_inertia = pd.DataFrame({ 'st rand':standard_rand_in, 'st plus':standard_plus_in,'min rand':mini_rand_in, 'min plus': mini_plus_in}, index= ['IRQ','LOF','Z-score'])
results_inertia


In [ ]:
#DB results(Values closer to zero indicate a better partition.)
results = pd.DataFrame({ 'st rand':standard_rand_dav, 'st plus':standard_plus_dav,'min rand':mini_rand_dav, 'min plus': mini_plus_dav}, index= ['IRQ','LOF','Z-score'])
results

In [ ]:
#sil results (want higher=highly dense clustering)
results = pd.DataFrame({ 'st rand':standard_rand_sil, 'st plus':standard_plus_sil,'min rand':mini_rand_sil, 'min plus': mini_plus_sil}, index= ['IRQ','LOF','Z-score'])
results

In [ ]:
#cal results(want higher)
results = pd.DataFrame({ 'st rand':standard_rand_cal, 'st plus':standard_plus_cal,'min rand':mini_rand_cal, 'min plus': mini_plus_cal}, index= ['IRQ','LOF','Z-score'])
results

## Result visualization 
Visulaizing the results of the nest method determined above.

LOF outlier ommision, MinMax scaling, elbow method k determination, k-means++ initalization. Independant od Clustering performed above.

In [ ]:
#data
data = lof_data
data.describe()

In [ ]:
#merging original data with cleaned data for plotting realtionships later
lof_data_full = pd.merge(data, ers_sample_records, right_index=True, left_index =True) 
lof_data_full
lof_data_full = lof_data_full.drop(columns=['MainWallIns_x'])
#re-setting index for mergeing with cluster lables later
data_all = lof_data_full
data_all = data_all.reset_index(drop=True)
data_all


In [ ]:
# scatter plot before clustering to look for intutive clusters 
sns.pairplot(data_all)

In [ ]:
#scaling with MinMax scaler
minmax = MinMaxScaler()
scaled_features = minmax.fit_transform(data)
#scaled feature into a data frame 
scaled_features = pd.DataFrame(scaled_features)

In [ ]:
#compare Stats for scaled and unsclaed data
#show scaled stats
scaled_features.describe()

In [ ]:
#show unscaled stats
data.describe()

In [ ]:
#plot histogram of scaled feaures 
scaled_features.hist(bins=10)

In [ ]:
# before scaling histogram
data.hist(bins=10)

In [ ]:
#clustering

kmeans_kwargs = {
    "init":"k-means++", 
    "n_init":10, 
    "max_iter":300,
    "random_state":42,}

sse=[] #determine SSE for 1 to 11 clusters    
for k in range(1,11): 
        kmeans = KMeans(n_clusters=k, **kmeans_kwargs)
        kmeans.fit(scaled_features)
        sse.append(kmeans.inertia_)
        
#plot elbow method
plt.style.use("fivethirtyeight")
plt.plot(range(1,11), sse)
plt.xticks(range(1,11))
plt.xlabel("Number of Clusters")
plt.ylabel("SSE")
plt.show()
        
k1 = KneeLocator(range(1,11),sse,curve="convex", direction="decreasing")
clamount=k1.elbow
    
#cluster with k determined above 
    
kmeans = KMeans(
    init="k-means++",
    n_clusters= clamount,
    n_init=10,
    max_iter=300,
    random_state=42)

In [ ]:
# create a dataframe including all variabels and cluster labels 
kmeans.fit(scaled_features)
labels = pd.DataFrame(kmeans.labels_,columns=['cluster label'])

cluster1=pd.concat([labels,data_all], axis = 1) 

cluster1

Looking at clusters individually 

In [ ]:
#cluster 0 dataframe
cluster_0 = cluster1.loc[cluster1['cluster label'] == 0, ['MainWallIns_y']]
cluster_0 = cluster_0.rename(columns={'MainWallIns_y': 'Cluster 0'})
# can view boxplot and stats separate but easier to compare on same table see below 
#cluster_0.boxplot()
#cluster_0.describe()

In [ ]:
#cluster 1 dataframe
cluster_1 = cluster1.loc[cluster1['cluster label'] == 1, ['MainWallIns_y']]
cluster_1=cluster_1.rename(columns={'MainWallIns_y': 'Cluster 1'})
#cluster_1.boxplot()
#cluster_1.describe()

In [ ]:
#cluster 2 dataframe
cluster_2 = cluster1.loc[cluster1['cluster label'] == 2, ['MainWallIns_y']]
#cluster_2.boxplot()
#cluster_2.describe()

In [ ]:
#cancatinate blox plot for each cluster to compare variation they capture for the certain vairbale 
#concatinate into a data frame 
all_clust = pd.concat([cluster_0,cluster_1, cluster_2], ignore_index=True, axis=1)
#plot using pandas 
all_clust.boxplot()

In [ ]:
#comparison of all cluster stats 
all_clust.describe()

In [ ]:
#Cluser Centroids 

centroids = minmax.inverse_transform(kmeans.cluster_centers_) # transform scaled cenroids back

centroids

In [ ]:
#Scatter plot of custers not scaled 
sns.scatterplot('MainWallIns_y', 'YearBuilt', data=cluster1, hue= 'cluster label')

In [ ]:
#scatter plot of scaled values 

# create a dataframe including all variabels and cluster labels 
kmeans.fit(scaled_features)
labels = pd.DataFrame(kmeans.labels_,columns=['cluster label'])

data_all_scaled = pd.DataFrame(minmax.fit_transform(data_all), columns = [ 'Air50P', 'Rating', 'YearBuilt', 'FloorArea',
       'MainWallIns_y'])

clusters_scaled = pd.concat([labels, data_all_scaled], axis = 1) 
sns.pairplot(clusters_scaled, hue = 'cluster label')

In [ ]:
# plot all varibles (not scaled) against eachother to find patterns in clusters  
sns.pairplot(cluster1, hue='cluster label')

In [ ]:
#silhouette coefficent visualizer
from yellowbrick.cluster import SilhouetteVisualizer

# Instantiate the clustering model and visualizer

visualizer = SilhouetteVisualizer(kmeans, colors='yellowbrick')

visualizer.fit(scaled_features)        # Fit the data to the visualizer
visualizer.show()        # Finalize and render the figure
#In SilhouetteVisualizer plots, clusters with higher scores have wider silhouettes, but clusters 
#that are less cohesive will fall short of the average score across all clusters, which is plotted as a
#vertical dotted red line.

In [ ]:
#inter cluster distance maps
from yellowbrick.cluster import InterclusterDistance

visualizer = InterclusterDistance(kmeans)

visualizer.fit(scaled_features)    # Fit the data to the visualizer
visualizer.show()        # Finalize and render the figure
#the closer to centers are in the visualization, the closer they are in the original feature space.

## 2D clustering 
This area is completely separate from above 1D clustering.

### Data preping 
Fist the data is retrieved and preped. Missing data is removed because k-means does not accept missing values and outliers are also removed because K-means is sensitive to outliers (a mean is easily influenced by extreme value). The preping will help acheive more robust clusting results. 

In [ ]:
#retreive data
ers_sample_records = pd.read_csv(r"C:\Users\owner\Documents\NRCan\code\practice\InitialHousingData.csv",)
ers_sample_records 

#original amount of obsevations/rows 
og_obs = ers_sample_records.shape[0]

In [ ]:
#retrive data from above
ers_sample_records


Two variables are selected fo perform clustering on. 

In [ ]:
#select variables for 2D clustering 
cl_variables = ['MainWallIns', 'Air50P' ] # for automation of code (hasn't been done yet)
twoD_data = ers_sample_records[cl_variables] # a varible towD_data is created to hold clustering varibles specifcally for 2D analysis 
twoD_data

In [ ]:
#review raw data stats 
twoD_data.describe()

Cleaning data. Dropna from pandas is used to remove rows were at least on element is na and local outlier factor is applied for outlier detection. 


In [ ]:
#Drop the rows where at least one element is NA.
twoD_data_clean = twoD_data.dropna()
removed = len(twoD_data)-len(twoD_data_clean)
#print amount of rows removed 
print('rows of data dropped:%d' %removed )

Local outlier factor (LOF) values identify an outlier based on the local neighborhood. It gives better results than the global approach to find outliers. A point will be considered as an outlier if it is at a small distance to the extremely dense cluster. 

In [ ]:
#plot varation with box plots before outlier removal to visualize outliers 
twoD_data.boxplot()

In [ ]:
#define the model

lof = LocalOutlierFactor()
lof_pred = lof.fit_predict(twoD_data_clean) 

#extract the negative outputs as the outliers.
mask = lof_pred != -1

#remove rows with outliers 
lof_data = twoD_data_clean[mask] #i dont think this works for sets with more than 1 varibale

#print amount of points deleated

outliers_rem = og_obs - lof_data.shape[0]
print ('amount of outliers removed: %d' %outliers_rem )

#plot without after removal to visualize outlier removal

lof_data.boxplot() 

#should scale and plot as they have very differnt scales 

Also can visualize outlier remvial though a scatter plot. The red values are the outliers that have been removed. 

In [ ]:
# create a dataframe of removed outliers for plotting 
from numpy import where
lofs_index = where(lof_pred==-1) 
#Filter_df  = twoD_data_clean[twoD_data_clean.index.isin(lofs_index)]
outliers =twoD_data_clean.loc[lofs_index] #datarfame of outliers 
#could write a check here: len of oultleris == outliers rem

In [ ]:
#plot removed outliers in red
plt.scatter(twoD_data_clean['MainWallIns'], twoD_data_clean['Air50P'])
plt.scatter(outliers['MainWallIns'],outliers['Air50P'], color='r')
plt.show()

Compare cleaned data stats to raw data to confirm the data is still meaningful. 

In [ ]:
#stats after outlier removal 
lof_data.describe()

In [ ]:
twoD_data.describe()

### Scaling 
k-means is distance based so for it to consider all attributes as equal and produce unbiased resutls, they must all have the same scale.

Min-max scaling rescales the data into a given range, in this case 0-1. 

In [ ]:
minmax = MinMaxScaler()
scaled_features = minmax.fit_transform(lof_data)

### Clustering 
In this step the preped data is clustered with the k-means algorithm. K-means++ is used for inialization and the amount of clusters is determined with internal validity measures.

In [ ]:
from yellowbrick.cluster import KElbowVisualizer
# Instantiate the clustering model and visualizer
# within cluster SSe (aka distortion)
#Distortion is the average of the euclidean squared distance from the centroid of the respective clusters. 
#Inertia is the sum of squared distances of samples to their closest cluster centre
model = KMeans(
    init="k-means++",
    n_init=10,
    max_iter=300,
    random_state=42)

#plot elbow method with distortion score

visualizer = KElbowVisualizer(model, k=(1,10))

visualizer.fit(lof_data)        # Fit the data to the visualizer
visualizer.show()        # Finalize and render the figure

#plot elbow method Calinski Harabasz 

visualizer = KElbowVisualizer(model, k=(2,10),metric='calinski_harabasz')

visualizer.fit(lof_data)        # Fit the data to the visualizer
visualizer.show()    

#plot elbow method with silhouette score 

visualizer = KElbowVisualizer(model, k=(2,10),metric='silhouette')

visualizer.fit(lof_data)        # Fit the data to the visualizer
visualizer.show()    

In [ ]:
# plot elbow method with interia 

sse=[] #determine SSE for 1 to 11 clusters
kmeans_kwargs = {
    "init":"k-means++",
    "n_init":10, 
    "max_iter":300,
    "random_state":42,
}

for k in range(1,11):
    kmeans = KMeans(n_clusters=k, **kmeans_kwargs)
    kmeans.fit(lof_data)
    sse.append(kmeans.inertia_)

plt.style.use("fivethirtyeight")
plt.plot(range(1,11), sse)
plt.xticks(range(1,11))
plt.xlabel("Number of Clusters")
plt.ylabel("SSE")
plt.show()

In [ ]:
#find the elbow/k
k1 = KneeLocator(
    range(1,11),sse,curve="convex", direction="decreasing")
k1.elbow

After the optimal k vlaue the clustering algortihm is ran once more and resulting clusters can be analyzed. 

In [ ]:
#final k-means arguments 
kmeans = KMeans(
    init="k-means++",
    n_clusters=6,
    n_init=10,
    max_iter=300,
    random_state=42
)

kmeans.fit(scaled_features)
labels = pd.DataFrame(kmeans.labels_,columns=['label']) #creating a dataframe to house cluster labels for each house
centroids = kmeans.cluster_centers_ #creating varible to hold centroid values 


In [ ]:
lof_data_new_index = lof_data.reset_index(drop=True) #new index to allow concatination to line up properly 

clusters = pd.concat([labels,lof_data_new_index], axis = 1) #can concatanate along columns but enssure that indicies are the same
#using groupby to compare clustering results 
label_group = clusters.groupby(['label']) #make varible for lable groups

label_group.get_group(1) #retrieve data frame of group

In [ ]:
#using group by to get median and means(cenroids) of the resulting clusters 
label_group['Air50P'].agg(['mean','median'])

In [ ]:
#geting mean and median for just one cluster 
label_group['MainWallIns'].agg(['mean','median'])

In [ ]:
#stats for all clusters by varible 
label_group.describe() 

In [ ]:
#scaling data for plotting
scaled_clusters = clusters.copy() #create a copy so that origonal not lost or modified 
scaled_clusters[['MainWallIns','Air50P']] = minmax.fit_transform(scaled_clusters[['MainWallIns','Air50P']]) #put in the cl_varibles or whatever to make it automated
scaled_clusters

In [ ]:
#centroids values 

In [ ]:
#plot with centroids
ax=sns.scatterplot(x='MainWallIns',y= 'Air50P', data=scaled_clusters, hue= 'label')
ax=sns.scatterplot(y=centroids[:,1],x=centroids[:,0], s=20, ec='black', legend=False, ax=ax)
plt.show()

In [ ]:
#plot without centroids and  not scaled 
sns.scatterplot(x='MainWallIns',y= 'Air50P', data=clusters, hue= 'label')
